<a href="https://colab.research.google.com/github/MrShaik7/Readwise-AI/blob/main/Resume_Interviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q chromadb sentence-transformers langchain pymupdf

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF
pdf_path = "/content/Obito.pdf"  # Replace with uploaded filename
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

# Extract only the text content from documents
texts = [chunk.page_content for chunk in chunks]
print(f"Loaded {len(texts)} chunks.")


Loaded 9 chunks.


In [ ]:
from sentence_transformers import SentenceTransformer

# Load local model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed all chunks
embeddings = embedding_model.encode(texts, convert_to_numpy=True)
print(embeddings)

[[ 0.0313511   0.02763218 -0.03671879 ...  0.016685    0.00253652
  -0.00023822]
 [-0.00323069 -0.00060114 -0.04327456 ...  0.03043053 -0.01892084
  -0.01733786]
 [ 0.02822407  0.08126036 -0.08495663 ...  0.02161446 -0.01176437
  -0.02483589]
 ...
 [ 0.02670352  0.09123499 -0.00824031 ...  0.05057378 -0.03555797
  -0.07717964]
 [ 0.00370365  0.12238184 -0.00554004 ...  0.03836922  0.04381477
  -0.02464417]
 [ 0.06595345  0.13885531  0.00420403 ... -0.031712    0.01905151
  -0.03286723]]


In [ ]:
import chromadb
from chromadb.config import Settings

# Initialize Chroma in-memory
chroma_client = chromadb.Client(Settings())

collection = chroma_client.get_or_create_collection(name="local_pdf_embeddings")

collection.add(
    documents=texts,
    embeddings=embeddings.tolist(),
    ids=[f"chunk_{i}" for i in range(len(texts))]
)
print("Chunks stored in ChromaDB.")

Chunks stored in ChromaDB.


In [ ]:
query = "What is the main character name?"

query_embedding = embedding_model.encode(query).tolist()

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

top_chunks = results['documents'][0]
context = "\n\n".join(top_chunks)

print("🧠 Retrieved context:\n")
print(context[:500] + "\n...")


🧠 Retrieved context:

dream through the Infinite Tsukuyomi.
 
Despite his actions, Obito remains a deeply conflicted character. His inner turmoil and
...
